In [18]:
import pandas as pd
import numpy as np

# Importação e Cópia
Estou importando os dados diretamente do Gsheet, e na célula seguinte fazendo a cópia dos dataframes. A cópia serve para não ter o trabalho de importar dataframe da nuvem novamente.

In [19]:
DF_COMPRAS_2024 = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/117UmGO31yywC6mucCwMFPmnQcg1kRI5WVRR1Cvpb6l8/export?format=xlsx&id=117UmGO31yywC6mucCwMFPmnQcg1kRI5WVRR1Cvpb6l8",
    sheet_name="Lançamento de NF Não Recorrente",
    usecols=[
        "Marca",
        "Unidade",
        "Item",
        "Valor Corrigido",
        "Mês Corrigido",
    ]
)

DF_DE_PARA_OFC = pd.read_excel(
    'https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=format=xlsx&id=1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ', 
    sheet_name="Unidades",
    usecols=[
        "Chave 2024",
        "Marca 2024",
        "Unidade 2024"
    ]
)

c:\Users\fernando.luiz\AppData\Local\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning: Cell E1954 is marked as a date but the serial value 6692727.0 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


In [37]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024.copy()
DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC.copy()

# Correção do Mês
Aqui estou covertendo os meses faltantes pelo mês atual, e não só isso, convertendo os valores para inteiro.

In [38]:
from datetime import datetime

DF_COMPRAS_2024_COPY["Mês Corrigido"] = pd.to_numeric(DF_COMPRAS_2024_COPY["Mês Corrigido"], "coerce")
DF_COMPRAS_2024_COPY["Mês Corrigido"] = DF_COMPRAS_2024_COPY["Mês Corrigido"].fillna(0)
DF_COMPRAS_2024_COPY["Mês Corrigido"] = DF_COMPRAS_2024_COPY["Mês Corrigido"].astype("int")
DF_COMPRAS_2024_COPY.loc[DF_COMPRAS_2024_COPY["Mês Corrigido"] == 0, "Mês Corrigido"] = datetime.now().month

# Correção de Marca e Unidade
Agora, vamos atualizar a marca e unidade para os nomes de 2024, nesse processo estou fazendo:
- Retirandos os dados faltantes
- Aplicando função ```stip()``` na Marca e Unidade, para retirar os espaços em branco
- Criando uma coluna apenas com a Unidade
- Criando a Chave 2024
- Aplicando o DePara e puxando as Marcas e Unidades atualizadas

In [39]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["Marca"].notna()]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["Unidade"].notna()]

DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC_COPY[DF_DE_PARA_OFC_COPY["Chave 2024"].notna()]
DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC_COPY.drop_duplicates()

DF_COMPRAS_2024_COPY["Marca"] = DF_COMPRAS_2024_COPY.apply(lambda x: x["Marca"].strip(), axis=1)
DF_COMPRAS_2024_COPY["Unidade"] = DF_COMPRAS_2024_COPY.apply(lambda x: x["Unidade"].strip(), axis=1)

DF_COMPRAS_2024_COPY["Unidade Atualizada"] = DF_COMPRAS_2024_COPY.apply(lambda row: row["Unidade"].replace(f"{row['Marca']} ", ""), axis=1)
DF_COMPRAS_2024_COPY["Chave 2024"] = DF_COMPRAS_2024_COPY["Marca"] + DF_COMPRAS_2024_COPY["Unidade Atualizada"] 

DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY.loc[(DF_COMPRAS_2024_COPY["Marca"] != "NUs")]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY.loc[(DF_COMPRAS_2024_COPY["Marca"] != "Visita BE")]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY.loc[(DF_COMPRAS_2024_COPY["Marca"] != "Salta")]

DF_COMPRAS_2024_DE_PARA = pd.merge(DF_COMPRAS_2024_COPY, DF_DE_PARA_OFC_COPY, on="Chave 2024", how="left")

# Dividindo o DataFrame
Nesse momento estou divindo meu DataFrame em dois, um com os erros, no qual vou tratar e unir novamente ao dataset original.

In [40]:
DF_ERROS = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].isna()]
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].notna()]

In [41]:
DF_ERROS["Marca"] = DF_ERROS.apply(lambda x: x["Unidade"].split(" ")[0], axis=1)

DF_ERROS["Unidade Atualizada"] = DF_ERROS.apply(lambda row: row["Unidade"].replace(f"{row['Marca']} ", ""), axis=1)
DF_ERROS["Chave 2024"] = DF_ERROS["Marca"] + DF_ERROS["Unidade Atualizada"]
DF_ERROS.drop(columns=[
    "Marca 2024",
    "Unidade 2024",
], inplace=True)

DF_ERROS = pd.merge(DF_ERROS, DF_DE_PARA_OFC_COPY, on="Chave 2024", how="left")

# Unindo as Bases de Dados

In [42]:
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Marca 2024",
    "Unidade 2024",
    "Valor Corrigido",
    "Mês Corrigido",
    "Item",
]]

DF_COMPRAS_2024_DE_PARA["Ano"] = 2024
DF_COMPRAS_2024_DE_PARA["NO"] = "Ativos"
DF_COMPRAS_2024_DE_PARA["Categoria"] = "Supply"
DF_COMPRAS_2024_DE_PARA["Tipo Valor"] = "Capex"
DF_COMPRAS_2024_DE_PARA["Tipo"] = "Real"

DF_ERROS_ATUALIZADOS = DF_ERROS[DF_ERROS["Marca 2024"].notna()]
DF_ERROS_ATUALIZADOS = DF_ERROS_ATUALIZADOS[[
    "Marca 2024",
    "Unidade 2024",
    "Valor Corrigido",
    "Mês Corrigido",
    "Item",
]]

DF_ERROS_ATUALIZADOS["Ano"] = 2024
DF_ERROS_ATUALIZADOS["NO"] = "Ativos"
DF_ERROS_ATUALIZADOS["Categoria"] = "Supply"
DF_ERROS_ATUALIZADOS["Tipo Valor"] = "Capex"
DF_ERROS_ATUALIZADOS["Tipo"] = "Real"

DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Valor Corrigido",
    "Marca 2024",
    "Unidade 2024",
    "Ano",
    "NO",
    "Categoria",
    "Tipo Valor",
    "Tipo",
    "Mês Corrigido",
    "Item"
]]

DF_ERROS_ATUALIZADOS = DF_ERROS_ATUALIZADOS[[
    "Valor Corrigido",
    "Marca 2024",
    "Unidade 2024",
    "Ano",
    "NO",
    "Categoria",
    "Tipo Valor",
    "Tipo",
    "Mês Corrigido",
    "Item", 
]]

In [43]:
DF_ATIVOS = pd.concat([DF_COMPRAS_2024_DE_PARA, DF_ERROS_ATUALIZADOS], axis=0)
DF_ATIVOS["Valor Corrigido"].sum() 

10767630.23

In [52]:
DF_ATIVOS["Item"] = DF_ATIVOS["Item"].str.lower().str.strip()

In [53]:
for i in DF_ATIVOS["Item"].unique():
    print(i)

carteiras diversas
1 carteira pne / 100 carteira univer. bm2002
quadro gestão / mural de aviso / quadro 400x120
projetores epson
nan
bebedouro
projetor - goldentec (220v)
moveis
projetor epson
mesa totó
material esportivo
carteiras
microondas
carteiras escolares metadil
prateleira de livros
claviculário para 50 chaves cityquadros
quadro branco quadriculado 600x 120
lixeiras itatiba
casinha do tarzan
mesa e cadeiras de refeitorio adulto/ cadeira de escritório básica azul
estante de madeira
mesa toto
armarios e puff
eletrodomesticos
playground
quadro
mesa futmesa
mat eletro
mural de aviso
tapete personalizado
bandeira escola
amplificador e cafeteira
mdf
projetor goldentec 7000
bebedouro linha domestica
bebedourogalão
bebedouros de galão – bancada – opção 2/bebedouros de galão – piso
bandeiras
bebedouro industrial
resfriador de agua industrial 3
torneiras 127v
lixeiras
cadeiras
projetor benq
tatame
radio transmissor
ventilador
colchão impermeavel
mat esportivo
bomber party
jogos bl de mon